In [1]:
import pandas as pd
import glob

In [2]:
file_paths = glob.glob('IG_Sentiments/*.csv')

In [3]:
dfs = []  # List to store DataFrames
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

['IG_Sentiments\\IG_Sentiments_Guiness_1.csv',
 'IG_Sentiments\\IG_Sentiments_Guiness_2.csv',
 'IG_Sentiments\\IG_Sentiments_Tusker_1.csv',
 'IG_Sentiments\\IG_Sentiments_Tusker_2.csv',
 'IG_Sentiments\\IG_Sentiments_Tusker_3.csv',
 'IG_Sentiments\\IG_Sentiments_Tusker_4.csv',
 'IG_Sentiments\\IG_Sentiments_Tusker_5.csv',
 'IG_Sentiments\\IG_Sentiments_Tusker_6.csv',
 'IG_Sentiments\\IG_Sentiments_Tusker_OktobaFest.csv',
 'IG_Sentiments\\IG_Sentiments_White_Cap_1.csv',
 'IG_Sentiments\\IG_Sentiments_White_Cap_2.csv']